In [1]:
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
from tqdm import tqdm
import random

In [2]:
n_particles = 1
system = mm.System()
mass   = 14.007 * unit.amu # masa del átomo N
system.addParticle(mass)

force = mm.CustomExternalForce('Eo*(C*x^4-x^2)+M*x + (K/2.0)*(y^2+z^2)')
force.addGlobalParameter('K', 35 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('Eo', 6.0 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('C', 0.4 * 1/unit.angstrom**2)
force.addGlobalParameter('M', 0.25 * unit.kilocalories_per_mole/unit.angstrom)
force.addParticle(0, [])
system.addForce(force)

0

In [3]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temps = [300,315,330,345,360,375,390]
pressure    = None

step_size       = 0.01*unit.picoseconds
num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond

platform_name = 'CPU'  
platform = mm.Platform.getPlatformByName(platform_name)


for T in temps:
    integrator = mm.LangevinIntegrator(T, friction, step_size)

    times      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
    positions  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)
    velocities = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)
    potential_energies = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
    kinetic_energies   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)

    initial_positions  = np.zeros([n_particles, 3], np.float32) * unit.nanometers
    initial_velocities = None 

    context = mm.Context(system, integrator, platform)
    context.setPositions(initial_positions)
    context.setVelocitiesToTemperature(T)

    state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
    times[0] = state.getTime()
    positions[0] = state.getPositions()[0]
    velocities[0] = state.getVelocities()[0]
    kinetic_energies[0]=state.getKineticEnergy()
    potential_energies[0]=state.getPotentialEnergy()

    for ii in tqdm(range(num_steps_saved)):
        context.getIntegrator().step(saving_period)
        state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
        times[ii] = state.getTime()
        positions[ii] = state.getPositions()[0]
        velocities[ii] = state.getVelocities()[0]
        kinetic_energies[ii]=state.getKineticEnergy()
        potential_energies[ii]=state.getPotentialEnergy()
    contextos = []
    contextos.append(times)


100%|██████████| 200/200 [00:03<00:00, 55.66it/s]


In [5]:
print(contextos)

[Quantity(value=array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148.

In [7]:
replicas = input("Escribe el número de réplicas que deseas")
rango_T = float(input("Escribe el rango de temperatura y el incremento a usar"))

Escribe el número de réplicas que deseas 3
Escribe el rango de temperatura y el incremento a usar 300,310,5


ValueError: could not convert string to float: '300,310,5'

In [4]:
print(rango_T)

300,400,25


In [6]:
np.arange(rango_T)

TypeError: unsupported operand type(s) for -: 'str' and 'int'